In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from mll_calc.mll_pred import format_XY, ratios, mll_testset

In [2]:
unc = 0.05
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
tamu_list = ['cs137/cs133', 'cs134/cs137', 'cs135/cs137', 'ba136/ba138',
             'sm150/sm149', 'sm152/sm149', 'eu154/eu153', 'pu240/pu239',
             'pu241/pu239', 'pu242/pu239']

In [3]:
train_db = '~/sims_n_results/sfcompo_compare_aug2020/van2ohi2_not-scaled_15nuc_5.pkl'
test_db = '~/sfcompo/format_clean/sfcompo_formatted.pkl'
XY = format_XY(train_db)
test = pd.read_pickle(test_db)

ratio_list = tamu_list
XY = ratios(XY, ratio_list, lbls)
test = ratios(test, ratio_list, lbls)

## Test sample from SFCOMPO that was simulated in ORIGEN

In [4]:
test = test.loc[test.index == 'VAN-2|EF05|WZR0058|E58-88']
test

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
VAN-2|EF05|WZR0058|E58-88,1.021429,0.012867,0.548252,0.0,97.640118,36.873156,0.124812,0.390879,0.184039,0.090554,pwr,0.0,4.5,43520.0,Vandellos-2_PWR


## VAN-2|EF05|WZR0058|E58-88 Simulation Parameters:
* PWR: Vandellos-2_PWR, so origen equivalent is W(17x17)
* Enrichment: 4.5
* Burnup: 43520.0 MW*d/tHM
* Cooling time: 2000 - 4000 days
* U weight: 72.45 MT (Did 1 MTU in sim)
* Thermal power: 2940 MW for cycles 7-10, and 3070 MW for cycle 11
* Avg power for ORIGEN-ARP is 40.58 & 42.37 MW/MTHM-- did avg of 41 for sim

In [5]:
XY[0:6]

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
0,1.104905,0.114118,0.326079,0.014627,131.863068,41.178407,0.238885,0.287179,0.335385,0.128256,pwr,0.0,4.5,43520.0,w17x17
1,0.966216,0.020601,0.370350,0.015046,91.086858,28.444748,0.152608,0.285016,0.253497,0.126462,pwr,2000.0,4.5,43520.0,w17x17
2,0.935811,0.013437,0.382383,0.015046,91.086858,28.444748,0.136641,0.285570,0.237357,0.126483,pwr,2500.0,4.5,43520.0,w17x17
3,0.906757,0.008756,0.394635,0.015046,91.086858,28.444748,0.122392,0.285907,0.222016,0.126483,pwr,3000.0,4.5,43520.0,w17x17
4,0.879054,0.005706,0.407071,0.015046,91.086858,28.444748,0.109606,0.286244,0.207856,0.126483,pwr,3500.0,4.5,43520.0,w17x17
5,0.851351,0.003721,0.420317,0.015046,91.086858,28.444748,0.098155,0.286581,0.194370,0.126483,pwr,4000.0,4.5,43520.0,w17x17


## First, try to find best prediction via MLL

In [6]:
pred_df = mll_testset(XY[0:6].copy(), test, True, unc, lbls)

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pred_df

,sim_idx,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,pred_idx,pred_ReactorType,pred_CoolingTime,pred_Enrichment,pred_Burnup,pred_OrigenReactor,MaxLogLL,MaxLLUnc,2ndMaxLogLL,2ndMaxLLUnc,CDF_LogLL_0.9998,CDF_LLUnc_0.9998,CDF_LogLL_0.9988,CDF_LLUnc_0.9988,CDF_LogLL_0.95,CDF_LLUnc_0.95,CDF_LogLL_0.9,CDF_LLUnc_0.9,CDF_LogLL_0.5,CDF_LLUnc_0.5,CDF_LogLL_0.1,CDF_LLUnc_0.1,CDF_LogLL_0.01,CDF_LLUnc_0.01
0,VAN-2|EF05|WZR0058|E58-88,pwr,0.0,4.5,43520.0,Vandellos-2_PWR,2,pwr,2500.0,4.5,43520.0,w17x17,-90.270654,24.088133,-121.709173,28.160071,-90.302092,178.254066,-90.459285,177.690869,-98.130283,150.206835,-105.989913,122.046964,-257.155033,33.48377,-837.155248,25.737647,-1241.349741,24.253084


## Second, dig in and look at ratio diffs

In [11]:
def logpdf_calc(sim_row, test_sample, unc):
    y_sim = sim_row[test_sample>0].values.tolist()
    std = sim_row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

In [12]:
test_sample = test.drop(columns=lbls).squeeze()
test_answer = test[lbls].squeeze()
sim_row = XY.loc[2].drop(lbls).squeeze()
logpdf = logpdf_calc(sim_row, test_sample, unc)
logpdf = pd.Series(logpdf, index=sim_row[test_sample>0].index, name='LogPDF')

In [13]:
df = pd.DataFrame([test_sample, sim_row])
df.loc['Abs Diff'] = np.abs(df.loc[2] - df.loc['VAN-2|EF05|WZR0058|E58-88'])
df.loc['% Diff'] = df.loc['Abs Diff'] * 100 / df.loc['VAN-2|EF05|WZR0058|E58-88']

In [14]:
df.loc['LogPDF'] = logpdf

In [15]:
#5 honed in cools
df

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
VAN-2|EF05|WZR0058|E58-88,1.021429,0.012867,0.548252,0.000000,97.640118,36.873156,0.124812,0.390879,0.184039,0.090554
2,0.935811,0.013437,0.382383,0.015046,91.086858,28.444748,0.136641,0.285570,0.237357,0.126483
Abs Diff,0.085618,0.000570,0.165869,0.015046,6.553260,8.428409,0.011829,0.105310,0.053318,0.035930
% Diff,8.382158,4.427484,30.254181,inf,6.711647,22.857844,9.477605,26.941728,28.970814,39.677799
LogPDF,0.469035,6.027037,-34.594492,NaN,-3.470242,-18.830866,2.568277,-23.868260,-6.576812,-11.994331
